## Deliverable 2. Create a Customer Travel Destinations Map.

In [85]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rafai,CF,4.9500,23.9167,69.21,87,98,1.25,overcast clouds
1,1,Thompson,CA,55.7435,-97.8558,60.96,77,75,0.00,broken clouds
2,2,Tual,ID,-5.6667,132.7500,80.98,82,99,12.80,overcast clouds
3,3,Avarua,CK,-21.2078,-159.7750,77.05,65,16,3.44,few clouds
4,4,Calama,CL,-22.4667,-68.9333,34.29,12,1,16.11,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_pref_temp = int(input("What is the minimum temperature you would like?"))
max_pref_temp = int(input("What is the maximum temperature you would like?"))

What is the minimum temperature you would like?54
What is the maximum temperature you would like?79


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
pref_temp_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_pref_temp) & 
                                (city_data_df["Max Temp"] <= max_pref_temp)]
pref_temp_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rafai,CF,4.9500,23.9167,69.21,87,98,1.25,overcast clouds
1,1,Thompson,CA,55.7435,-97.8558,60.96,77,75,0.00,broken clouds
3,3,Avarua,CK,-21.2078,-159.7750,77.05,65,16,3.44,few clouds
5,5,Vaini,TO,-21.2000,-175.2000,69.96,78,75,10.36,broken clouds
8,8,Esperance,AU,-33.8667,121.9000,58.84,49,4,18.95,clear sky
9,9,Isangel,VU,-19.5500,169.2667,78.91,53,40,11.50,scattered clouds
10,10,Aklavik,CA,68.2191,-135.0107,59.83,73,99,11.61,overcast clouds
14,14,Blackwater,AU,-23.5833,148.8833,77.05,37,0,8.81,clear sky
15,15,Jamestown,US,42.0970,-79.2353,67.71,88,75,2.62,broken clouds
17,17,Teya,RU,60.3778,92.6267,62.20,60,100,7.16,overcast clouds


In [19]:
# 4a. Determine if there are any empty rows.
pref_temp_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_temp_df = pref_temp_df.dropna()
pref_temp_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [81]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_temp_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
test_hotel_df = hotel_df.head(5)
test_hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rafai,CF,69.21,overcast clouds,4.9500,23.9167,
1,Thompson,CA,60.96,broken clouds,55.7435,-97.8558,
3,Avarua,CK,77.05,few clouds,-21.2078,-159.7750,
5,Vaini,TO,69.96,broken clouds,-21.2000,-175.2000,
8,Esperance,AU,58.84,clear sky,-33.8667,121.9000,


In [88]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in test_hotel_df.iterrows():
    
    try:
        
        # 6c. Get latitude and longitude from DataFrame
        lat = row["Lat"]
        lng = row["Lng"]
        
        params['location'] = f"{lat},{lng}"

        # 6d. Set up the base URL for the Google Directions API to get JSON data.
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

        # 6e. Make request and retrieve the JSON data from the search.
        hotels = requests.get(url, params=params).json()

        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except:
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("No hotel found. Skipping city...")


No hotel found. Skipping city...


In [94]:
hotel_df["Hotel Name"].dropna()
hotel_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rafai,CF,69.21,overcast clouds,4.9500,23.9167,NaN
1,Thompson,CA,60.96,broken clouds,55.7435,-97.8558,Burntwood Hotel
3,Avarua,CK,77.05,few clouds,-21.2078,-159.7750,Paradise Inn
5,Vaini,TO,69.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
8,Esperance,AU,58.84,clear sky,-33.8667,121.9000,Hospitality Esperance
9,Isangel,VU,78.91,scattered clouds,-19.5500,169.2667,Tanna Lodge
10,Aklavik,CA,59.83,overcast clouds,68.2191,-135.0107,Aklavik Inn
14,Blackwater,AU,77.05,clear sky,-23.5833,148.8833,Blackwater Motor Inn
15,Jamestown,US,67.71,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
17,Teya,RU,62.20,overcast clouds,60.3778,92.6267,None


In [74]:
# import pprint as pp
# pp.pprint(hotels)

In [9]:
# 7. Drop the rows where there is no Hotel Name.


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
